In [1]:
import pandas as pd
import numpy as np

# 1. Inlezen
df = pd.read_csv("flowrate_basin.csv", parse_dates=["Time"])
naam_kolom = df.columns[1]

# 2. Opschonen (optioneel, alleen als je al outliers wil filteren)
df = df.dropna()
rolling_median = df[naam_kolom].rolling(window=700, center=True).median()
diff = np.abs(df[naam_kolom] - rolling_median)
threshold = 0.1
df = df[diff < threshold]

# 3. Zet tijd als index en sorteer
df = df.set_index("Time").sort_index()

# 4. Maak tijdraster op exact 10-minutenintervallen
start = df.index.min().floor("10min")
end = df.index.max().ceil("10min")
time_grid = pd.date_range(start=start, end=end, freq="10min")

# 5. Nieuwe dataframe met enkel de grid
df_grid = pd.DataFrame(index=time_grid)

# 6. Voeg originele data toe en interpoleer
df_full = pd.concat([df, df_grid], axis=0).sort_index()
df_interp = df_full.interpolate(method="time")

# 7. Selecteer enkel waarden exact op 10-minuten
df_final = df_interp.loc[time_grid].reset_index().rename(columns={"index": "Time"})

# Zorg dat 'Time' kolom datetime is (zou het al moeten zijn na voorgaande code)
df_final["Time"] = pd.to_datetime(df_final["Time"])

# Filter enkel waarden vanaf (inclusief) 25 oktober 2023
df_filtered = df_final[(df_final["Time"] >= pd.Timestamp("2023-10-25")) & (df_final["Time"] <= pd.Timestamp("2025-02-19"))]
df_filtered = df_filtered.rename(columns={"Time": "Datetime"})

# (Optioneel) Opslaan
df_filtered.to_csv("flowrate_interpolated.csv", index=False)
